In [4]:
import importlib
import GUI_notebook
importlib.reload(GUI_notebook)
vb = GUI_notebook.init(In)
vb

Box(children=(Box(children=(Btn(children=['Copy cells'], color='blue lighten-1', height='35px', width='250px')…

```python
import numpy as np

class Component1(om.ExplicitComponent):

	def setup(self):
		self.add_input('x_name', val=np.nan)
		self.add_output('y_name')
		self.add_output('z_name')

	def setup_partials(self):
		self.declare_partials('y_name', 'x_name')
		self.declare_partials('z_name', ['y_name', 'x_name'])

	def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):

		x = inputs['x_name']

		y = x + 3*(
		x**2 + 3
		)
		z = y**3 + 2*(
		x**2 + 3
		)

		outputs['y_name'] = y
		outputs['z_name'] = z


	def compute_partials(self, inputs, J):

		x = inputs['x_name']

		J['y_name','x_name'] = 6*x + 1

		J['z_name','y_name'] = 3*y**2
		J['z_name','x_name'] = 4*x


```

In [ ]:
"""
---Test Text---

# Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

# Component2

long_variable = a + b #[long_unit]
function_test = np.log(x + np.sin(y)) #[rad]


"""

In [ ]:
"""
---Test Text---
## Group1

# Component1

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]

## Group2

# Component3

x = 2*y + 3*z
a = b + c

# Component4

long_variable = a + b
function_test = np.log(x + np.sin(y))

"""

In [ ]:
# Init
x = 0
z = 1

In [ ]:
# Exclude
"""
some commentary
"""

In [ ]:
### HG1

## Group1

# Component1
y = x + 1
y = y**2 + z

In [ ]:
# Exclude
y

In [ ]:
# Init
b = 3
c = 4

In [ ]:
### HG2
## Group2

# Component2

x = 2*y + 3*z #[m]
a = b + c*1.2e-3 #[kg]

In [ ]:
# Exclude
import numpy as np
a = 0
b = 1

In [ ]:
# Component3

long_variable = a + b
function_test = np.log(x + np.sin(y)) #[rad]